In [29]:
import h2o
import pandas
import pprint
import operator
import matplotlib
from tabulate import tabulate

In [30]:
# Connect to a cluster
h2o.init()

H2O cluster uptime:,31 seconds 66 milliseconds
H2O cluster version:,3.1.0.99999
H2O cluster name:,spIdea
H2O cluster total nodes:,1
H2O cluster total memory:,12.44 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [31]:
# set this to True if interactive (matplotlib) plots are desired
interactive = False
if not interactive: matplotlib.use('Agg', warn=False)
import matplotlib.pyplot as plt

In [32]:
# air_path = [h2o.locate("bigdata/laptop/airlines_all.05p.csv")]
# air_path = [h2o.locate("bigdata/laptop/flights-nyc/flights14.csv.zip")]
air_path = [h2o.locate("smalldata/airlines/allyears2k_headers.zip")]

# ----------

# 1- Load data - 1 row per flight.  Has columns showing the origin,
# destination, departure and arrival time, carrier information, and
# whether the flight was delayed.
print "Import and Parse airlines data"
data = h2o.import_file(path=air_path)
data.describe()

Import and Parse airlines data

Parse Progress: [##################################################] 100%

Parsed 43,978 rows and 31 cols:



File1,/Users/spencer/0xdata/h2o-3/smalldata/airlines/allyears2k_headers.zip


Rows: 43,978 Cols: 31
  chunk_type                 chunk_name  count  count_percentage        size  \
0        C0L          Constant Integers     10          5.376344      800  B   
1        C0D             Constant Reals     23         12.365591      1.8 KB   
2        CBS                       Bits      2          1.075269      2.0 KB   
3        CX0                Sparse Bits     10          5.376344      1.9 KB   
4         C1            1-Byte Integers     40         21.505377    287.8 KB   
5        C1N  1-Byte Integers (w/o NAs)     19         10.215054    133.1 KB   
6        C1S           1-Byte Fractions      6          3.225807     43.4 KB   
7         C2            2-Byte Integers     76         40.860214      1.1 MB   

   size_percentage  
0         0.050402  
1         0.115925  
2         0.127203  
3         0.124746  
4        18.564957  
5         8.586170  
6         2.802498  
7        69.628105  
                            size  number_of_rows  number_of_chunks_p

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
type,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,int,enum,enum,int,int,int,int,enum,int,int,int,int,int,int,enum,enum
mins,1987.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,16.0,17.0,14.0,-63.0,-16.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
maxs,2008.0,10.0,31.0,7.0,2400.0,2359.0,2400.0,2359.0,9.0,3949.0,3500.0,475.0,437.0,402.0,475.0,473.0,131.0,133.0,3365.0,128.0,254.0,1.0,3.0,1.0,369.0,201.0,323.0,14.0,373.0,1.0,1.0
sigma,6.34436090171,1.87471137134,9.17579042586,1.90501311913,465.340899124,476.251139993,484.347487904,492.750434123,NaN,777.404369164,NaN,73.9744416606,73.40159463,69.6363295151,29.8402219624,26.4388090429,NaN,NaN,578.43800823,4.20197993986,9.9050857472,0.155193141358,NaN,0.0497234872189,16.2057299045,4.41677989873,18.6197762215,0.403940182102,23.4875658741,NaN,NaN
zero_count,0,0,0,0,0,569,0,569,724,0,2,0,0,0,1514,6393,59,172,0,623,557,42892,81,43869,7344,8840,7388,8914,7140,19537,20887
missing_count,0,0,0,0,1086,0,1195,0,0,0,32,1195,13,16649,1195,1086,0,0,35,16026,16024,0,9774,0,35045,35045,35045,35045,35045,0,0


In [33]:
# ----------

# 2- Data exploration and munging. Generate scatter plots 
# of various columns and plot fitted GLM model.

# Function to fit a GLM model and plot the fitted (x,y) values
def scatter_plot(data, x, y, max_points = 1000, fit = True):
    if(fit):
        lr = h2o.glm(x = data[[x]], y = data[y], family = "gaussian")
        coeff = lr.coef()
    df = data[[x,y]]
    runif = df[y].runif()
    df_subset = df[runif < float(max_points)/data.nrow]
    df_py = h2o.as_list(df_subset)
    
    if(fit): h2o.remove(lr._id)

    # If x variable is string, generate box-and-whisker plot
    if(df_py[x].dtype == "object"):
        if interactive: df_py.boxplot(column = y, by = x)
    # Otherwise, generate a scatter plot
    else:
        if interactive: df_py.plot(x = x, y = y, kind = "scatter")
    
    if(fit):
        x_min = min(df_py[x])
        x_max = max(df_py[x])
        y_min = coeff["Intercept"] + coeff[x]*x_min
        y_max = coeff["Intercept"] + coeff[x]*x_max
        plt.plot([x_min, x_max], [y_min, y_max], "k-")
    if interactive: plt.show()

scatter_plot(data, "Distance", "AirTime", fit = True)
scatter_plot(data, "UniqueCarrier", "ArrDelay", max_points = 5000, fit = False)


glm Model Build Progress: [##################################################] 100%


In [34]:
# Group flights by month
grouped = data.group_by("Month")
bpd = grouped.count().sum("Cancelled").frame
bpd.show()
bpd.describe()
bpd.dim

# Convert columns to factors
data["Year"]      = data["Year"]     .asfactor()
data["Month"]     = data["Month"]    .asfactor()
data["DayOfWeek"] = data["DayOfWeek"].asfactor()
data["Cancelled"] = data["Cancelled"].asfactor()

H2OFrame with 2 rows and 3 columns: 
   Month  sum_Cancelled  nrow_Year
0      1           1067      41979
1     10             19       1999
Rows: 2 Cols: 3
  chunk_type                 chunk_name  count  count_percentage        size  \
0        C1N  1-Byte Integers (w/o NAs)      1         33.333336       70  B   
1         C2            2-Byte Integers      1         33.333336       72  B   
2        C2S           2-Byte Fractions      1         33.333336       88  B   

   size_percentage  
0        30.434782  
1        31.304348  
2        38.260868  
                            size  number_of_rows  number_of_chunks_per_column  \
0  172.16.2.82:54321      230  B               2                            1   
1               mean      230  B               2                            1   
2                min      230  B               2                            1   
3                max      230  B               2                            1   
4             stddev        0  B

,Month,sum_Cancelled,nrow_Year
type,int,int,int
mins,1.0,19.0,1999.0
maxs,10.0,1067.0,41979.0
sigma,6.36396103068,741.047906684,28270.1291118
zero_count,0,0,0
missing_count,0,0,0


In [35]:
# Calculate and plot travel time
hour1 = data["CRSArrTime"] / 100
mins1 = data["CRSArrTime"] % 100
arrTime = hour1*60 + mins1

hour2 = data["CRSDepTime"] / 100
mins2 = data["CRSDepTime"] % 100
depTime = hour2*60 + mins2

# TODO: Replace this once list comprehension is supported. See PUBDEV-1286.
# data["TravelTime"] = [x if x > 0 else None for x in (arrTime - depTime)]
data["TravelTime"] = h2o.ifelse((arrTime-depTime) > 0, (arrTime-depTime), h2o.H2OFrame(python_obj=[[None] for r in range(data.nrow)]))[0]
scatter_plot(data, "Distance", "TravelTime")


Parse Progress: [##################################################] 100%
Uploaded pye2fccd5b-2d5f-4c5c-8ea8-d8d0fbd6c556 into cluster with 43,978 rows and 1 cols

glm Model Build Progress: [##################################################] 100%


In [36]:
# Impute missing travel times and re-plot
data.impute(column = "Distance", by = ["Origin", "Dest"])
scatter_plot(data, "Distance", "TravelTime")


glm Model Build Progress: [##################################################] 100%


In [37]:
# ----------
# 3- Fit a model on train; using test as validation

# Create test/train split
s = data["Year"].runif()
train = data[s <= 0.75]
test  = data[s > 0.75]

# Set predictor and response variables
myY = "IsDepDelayed"
myX = ["Origin", "Dest", "Year", "UniqueCarrier", "DayOfWeek", "Month", "Distance", "FlightNum"]

# Simple GLM - Predict Delays
data_glm = h2o.glm(x           =train[myX],
                   y           =train[myY],
                   validation_x=test [myX],
                   validation_y=test [myY],
                   family      ="binomial",
                   standardize =True)

# Simple GBM
data_gbm = h2o.gbm(x              =train[myX],
                   y              =train[myY],
                   validation_x   =test [myX],
                   validation_y   =test [myY],
                   balance_classes=True,
                   ntrees         =3,
                   max_depth      =1,
                   distribution   ="bernoulli",
                   learn_rate     =0.1,
                   min_rows       =2)

# Complex GBM
data_gbm2 = h2o.gbm(x              =train[myX],
                    y              =train[myY],
                    validation_x   =test [myX],
                    validation_y   =test [myY],
                    balance_classes=True,
                    ntrees         =50,
                    max_depth      =5,
                    distribution   ="bernoulli",
                    learn_rate     =0.1,
                    min_rows       =2)

# Simple Random Forest
data_rf = h2o.random_forest(x              =train[myX],
                            y              =train[myY],
                            validation_x   =test [myX],
                            validation_y   =test [myY],
                            ntrees         =5,
                            max_depth      =2,
                            balance_classes=True)

# Complex Random Forest
data_rf2 = h2o.random_forest(x              =train[myX], 
                             y              =train[myY],
                             validation_x   =test [myX],
                             validation_y   =test [myY],
                             ntrees         =10,
                             max_depth      =5,
                             balance_classes=True)

# Deep Learning with 5 epochs
data_dl = h2o.deeplearning(x                   =train[myX],
                           y                   =train[myY],
                           validation_x        =test [myX],
                           validation_y        =test [myY],
                           hidden              =[10,10],
                           epochs              =5,
                           variable_importances=True,
                           balance_classes     =True,
                           loss                ="Automatic")


glm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

drf Model Build Progress: [##################################################] 100%

drf Model Build Progress: [##################################################] 100%

deeplearning Model Build Progress: [##################################################] 100%


In [38]:
# Variable importances from each algorithm
# Calculate magnitude of normalized GLM coefficients
glm_varimp = data_glm.coef_norm()
for k,v in glm_varimp.iteritems():
    glm_varimp[k] = abs(glm_varimp[k])
    
# Sort in descending order by magnitude
glm_sorted = sorted(glm_varimp.items(), key = operator.itemgetter(1), reverse = True)
table = tabulate(glm_sorted, headers = ["Predictor", "Normalized Coefficient"], tablefmt = "orgtbl")
print "Variable Importances:\n\n" + table

data_gbm.varimp()
data_rf.varimp()

Variable Importances:

| Predictor        |   Normalized Coefficient |
|------------------+--------------------------|
| Year.2008        |               2.24728    |
| Origin.MDW       |               1.59547    |
| Year.2003        |               1.57759    |
| Dest.HTS         |               1.46753    |
| Year.2007        |               1.4584     |
| Dest.LYH         |               1.42202    |
| Origin.HPN       |               1.41025    |
| Origin.LIH       |               1.40548    |
| Origin.LEX       |               1.27632    |
| UniqueCarrier.HP |               1.21406    |
| Origin.TLH       |               1.21254    |
| Origin.CAE       |               1.21213    |
| Origin.TRI       |               1.20846    |
| Year.2001        |               1.03136    |
| Origin.HNL       |               1.03046    |
| Origin.LBB       |               0.989776   |
| UniqueCarrier.TW |               0.982729   |
| Origin.BOI       |               0.967928   |
| Origin.BTV     

In [ ]:
# Model performance of GBM model on test data
data_gbm2.model_performance(test)